In [0]:
ISOCHRONES_PATH = '/datascope/subaru/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc'
OBS_PATH = '/datascope/subaru/data/cmdfit/dSph/bootes_tpalle_g24.cat'
SIM_PATH = '/datascope/subaru/data/cmdfit/run/boo/sim/nobin_250k_006/sample.h5'

In [0]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from scipy.special import logsumexp
from scipy.interpolate import interp1d

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

# Load observations

In [0]:
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color

In [0]:
obs = SubaruHSC.text_observation_reader(mags=['r', 'g'], ext=['g', 'r', 'n']).read(OBS_PATH)
obs.data.head()

# Load simulation

In [0]:
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color

In [0]:
# Bootes
if True:
    hsc = SubaruHSC.photometry()
    cmd = CMD([
            ColorAxis(
                Color([hsc.magnitudes['g'], hsc.magnitudes['r']]),
                limits=(-1, 4)),
            MagnitudeAxis(
                hsc.magnitudes['g'],
                limits=(15.5, 24.5)
            )
        ])
    ccd = None
    # ccd = CCD([
    #     cmd.axes[0],
    #     ColorAxis(
    #         Color([hsc.magnitudes['g'], hsc.magnitudes['nb515']]),
    #         limits=(-0.5, 0.5)
    #     )
    # ])

In [0]:
from pfs.ga.targeting.io import Hdf5SimulationReader

In [0]:
r = Hdf5SimulationReader()
r.append_photometry(SubaruHSC.photometry())
sim = r.read(SIM_PATH)

In [0]:
for k in sim.data.keys():
    print(k, sim.data[k].shape)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_simulation(axs[0], sim, s=np.s_[::100], size=0.05)
# ccd.plot_simulation(axs[1], sim, s=np.s_[::100], size=0.05)

f.tight_layout()

# Create probability map

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling

In [0]:
# Fornax dwarf
if True:
    # Original weights
    print('data.w', sim.data['w'])

    w = np.bincount(sim.data['g']) / sim.data['g'].shape
    print('w', w, np.sum(w[:-2]), np.sum(w[-2:]))

    w1 = w.copy()
    w1[:-6] * 0.7       # MW foreground
    w1[-6:-4] *= 2.0    # broad RGB population
    w1[-4:-2] *= 0.3    # old RGB population
    w1[-2:] *= 1.0      # member MS population
    w1 /= w1.sum()

    g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
    print('g1', g1.shape)

    # Verify new categories
    w2 = np.bincount(g1) / g1.shape
    print('w2', w2, np.sum(w2[:-2]), np.sum(w2[-2:]))

In [0]:
# Bootes I
if True:
    # Weights found by eye
    w1 = np.array([0, 0, 0, 0.017, 0.014, 0.005])
    w1 /= w1.sum()

    print(w1)

In [0]:
# The simulation has 10 + 2 * N sub-populations (binaries treated separately for each)
# 10 is for 5 MW (3 for the thin disk + 1 thick disk + 1 halo)
# Merge foreground populations and members + member binaries when creating the map

# Ursa Minor
if False:
    extents = [[0.1, 2.0], [17.0, 23.5]]

# Fornax
if False:
    extents = [[-0.75, 2.0], [17.0, 23.5]]

# Bootes
if True:
    extents = [[0, 1.5], [17.5, 23.0]]

pmap = ProbabilityMap(cmd.axes)
pmap.from_simulation(sim, bins=[100, 100], extents=extents,
    # merge_list=[np.s_[:10], np.s_[10:]], population_weights=w1, observed=True, mask=None)
    merge_list=[np.s_[:5], np.s_[5:]], population_weights=w1, observed=True, mask=None)
pmap.maximum_filter()

In [0]:
pmap.extents

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

l0 = cmd.plot_probability_map(axs[0], pmap, 0)
l1 = cmd.plot_probability_map(axs[1], pmap, 1)

f.tight_layout()

# Membership probability based on the map

In [0]:
lp_member, mask_member = pmap.lookup_lp_member(obs)

lp_member.shape, np.isnan(lp_member).sum(), np.isnan(lp_member[mask_member]).sum(), mask_member.shape, mask_member.sum()

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs, c=lp_member[...,0])
# ccd.plot_observation(axs[1], obs, c=lp_member[...,0])

f.tight_layout()

# Selection based on probability cut

In [0]:
pcut = ProbabilityCut(pmap, 1, -3)  # cut at e**-3

In [0]:
mask = pcut.apply(obs, observed=True)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs, mask=mask)
# ccd.plot_observation(axs[1], obs, mask=mask)

f.tight_layout()